# Proyecto #7 - Análisis A/B Testing para un E-commerce con un DataSet de 1.600 Registros

Nuestra tienda de zapatos favorita en línea, ShoeFly.com está realizando un A/B testing. Tienen dos versiones diferentes de un anuncio, que han colocado en correos electrónicos, así como en banners en Facebook, Twitter y Google. Quieren saber cómo se están desempeñando los dos anuncios en cada una de las diferentes plataformas cada día de la semana. Ayúdelos a analizar los datos usando medidas agregadas.

### Análisis de las fuentes de anuncios

1. Examine las primeras filas de `ad_clicks`.

In [3]:
import pandas as pd

In [4]:
clicks = pd.read_csv('ad_clicks.csv')
print(clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


2. Su administrador desea saber qué plataforma publicitaria le está dando más visitas.
¿Cuántas vistas (es decir, filas de la tabla) provienen de cada fuente `utm_source`?

In [6]:
source_count = clicks.groupby('utm_source').count().reset_index()
print(source_count)

  utm_source  user_id  day  ad_click_timestamp  experimental_group
0      email      255  255                  80                 255
1   facebook      504  504                 180                 504
2     google      680  680                 239                 680
3    twitter      215  215                  66                 215


3. Si la columna `ad_click_timestamp` es not null, entonces alguien hizo clic en el anuncio que se mostró.
Cree una nueva columna llamada `is_click`, que es `True` si `ad_click_timestamp` es not null y `False` de otra manera.

In [12]:
clicks['is_click'] = clicks.ad_click_timestamp.isnull()
print(clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  is_click  
0               7:18                  A     False  
1                NaN                  B      True  
2                NaN                  A      True  
3                NaN                  B      True  
4                NaN                  B      True  


4. Queremos saber el porcentaje de personas que hicieron clic en los anuncios de cada `utm_source`.
Comience agrupando por `utm_source` e `is_click` y contando el número de `user_id`'s en cada uno de esos grupos. Guarde su respuesta en la variable `clicks_by_source`.

In [35]:
clicks_by_source = clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()
clicks_by_source

,utm_source,is_click,user_id
0,email,False,80
1,email,True,175
2,facebook,False,180
3,facebook,True,324
4,google,False,239
5,google,True,441
6,twitter,False,66
7,twitter,True,149


5. Ahora vamos a pivotar los datos para que las columnas sean `is_click` (`True` o `False`), el índice sea `utm_source`, y los valores sean `user_id`.
Guarde sus resultados en el variable `clicks_pivot`.

In [36]:
clicks_pivot = clicks_by_source.pivot(columns='is_click', index='utm_source', values='user_id').reset_index()
clicks_pivot

is_click,utm_source,False,True
0,email,80,175
1,facebook,180,324
2,google,239,441
3,twitter,66,149


6. Cree una nueva columna en `clicks_pivot` llamada `percent_clicked`, que es igual al porcentaje de usuarios que han hecho clic en el anuncio desde cada `utm_source.
¿Hubo alguna diferencia en las tasas de clics para cada fuente?

In [37]:
clicks_pivot['percent_clicked'] = clicks_pivot[True]/(clicks_pivot[True]+clicks_pivot[False])
clicks_pivot

is_click,utm_source,False,True,percent_clicked
0,email,80,175,0.686275
1,facebook,180,324,0.642857
2,google,239,441,0.648529
3,twitter,66,149,0.693023


### Analizar una A/B Testing

7. La columna `experimental_group` nos dice si al usuario le fue mostrado el anuncio A o anuncio B.
¿Se mostró aproximadamente al mismo número de personas en ambos anuncios?

In [16]:
exp_count = clicks.groupby('experimental_group').user_id.count().reset_index()
exp_count

,experimental_group,user_id
0,A,827
1,B,827


8. Usando la columna `is_click` que definimos anteriormente, comprobar si un mayor porcentaje de usuarios han hecho clic en Ad A o Ad B.

In [25]:
exp_percent_pivot = clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index().pivot(
    columns = 'is_click', index = 'experimental_group', values = 'user_id').reset_index()
print(exp_percent_pivot)
percent_exper = exp_percent_pivot[True]/(exp_percent_pivot[False]+exp_percent_pivot[True])
print(percent_exper)

is_click experimental_group  False  True
0                         A    310   517
1                         B    255   572
0    0.625151
1    0.691657
dtype: float64


9. El Gerente de Producto para el A/B Test piensa que los clics podrían haber cambiado en el día de la semana.
Comience por crear dos DataFrames: `a_clicks` y `b_clicks`, que contienen sólo los resultados de grupo `A` y grupo `B`, respectivamente.

In [30]:
a_clicks = clicks[clicks.experimental_group == 'A']
b_clicks = clicks[clicks.experimental_group == 'B']
print(a_clicks)
print(b_clicks)

                                   user_id utm_source            day  \
0     008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
2     00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
5     013b0072-7b72-40e7-b698-98b4d0c9967f   facebook     1 - Monday   
6     0153d85b-7660-4c39-92eb-1e1acd023280     google   4 - Thursday   
7     01555297-d6e6-49ae-aeba-1b196fdbb09f     google  3 - Wednesday   
...                                    ...        ...            ...   
1643  fceb13ea-fd8c-446a-a61f-f977d404330a    twitter   6 - Saturday   
1646  fd7d06ea-38b5-4ed9-acc9-777047db8c56     google   4 - Thursday   
1647  fe570a20-448f-40ed-930b-8482b8a7c231   facebook     1 - Monday   
1649  fe8b5236-78f6-4192-9da6-a76bba67cfe6    twitter     7 - Sunday   
1652  ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732     google     1 - Monday   

     ad_click_timestamp experimental_group  is_click  
0                  7:18                  A     False  
2                   NaN  

10. Para cada grupo (`a_clicks` y `b_clicks`), calcule el porcentaje de usuarios que hicieron clic en el anuncio por día.

In [43]:
a_clicks_count = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
    columns='is_click', 
    index='day', 
    values='user_id'
).reset_index()
a_clicks_count['percent_clicks'] = a_clicks_count[True]/(a_clicks_count[False]+a_clicks_count[True])
a_clicks_count

is_click,day,False,True,percent_clicks
0,1 - Monday,43,70,0.619469
1,2 - Tuesday,43,76,0.638655
2,3 - Wednesday,38,86,0.693548
3,4 - Thursday,47,69,0.594828
4,5 - Friday,51,77,0.601562
5,6 - Saturday,45,73,0.618644
6,7 - Sunday,43,66,0.605505


In [44]:
b_clicks_count = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
    columns='is_click', 
    index='day', 
    values='user_id'
).reset_index()
b_clicks_count['percent_clicks'] = b_clicks_count[True]/(b_clicks_count[False]+b_clicks_count[True])
b_clicks_count

is_click,day,False,True,percent_clicks
0,1 - Monday,32,81,0.716814
1,2 - Tuesday,45,74,0.621849
2,3 - Wednesday,35,89,0.717742
3,4 - Thursday,29,87,0.750000
4,5 - Friday,38,90,0.703125
5,6 - Saturday,42,76,0.644068
6,7 - Sunday,34,75,0.688073


11. Compare los resultados de `A` y `B`. ¿Qué pasó en el transcurso de la semana?
¿Recomienda que su empresa utilice el anuncio `A` o el anuncio `B`?

In [55]:
Mejor =  'Ad a' if (a_clicks_count.percent_clicks.mean() > b_clicks_count.percent_clicks.mean()) else 'Ad b'
Mejor

'Ad b'

In [ ]:
recommend = "ad A"